In [2]:
import pandas as pd



In [ ]:
#Load data from the clean CSV file
df = pd.read_csv(
    "../data/processed/online_retail_clean.csv",
    parse_dates=['InvoiceDate']
)

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [4]:
#Let's create a Revenue column
df['Revenue'] = df['Quantity'] * df['UnitPrice']

# Let's look at the first few rows of Quantity, UnitPrice, and Revenue
df[['Quantity', 'UnitPrice', 'Revenue']].head()



,Quantity,UnitPrice,Revenue
0,6,2.55,15.30
1,6,3.39,20.34
2,8,2.75,22.00
3,6,3.39,20.34
4,6,3.39,20.34


In [ ]:
# Let's find the most recent InvoiceDate to use as a reference date
# What this code does is to get the maximum date from the 'InvoiceDate' column
reference_date = df['InvoiceDate'].max()
reference_date


Timestamp('2011-12-09 12:50:00')

In [6]:
customer_df = (
    df.groupby('CustomerID')
      .agg(
          last_purchase_date=('InvoiceDate', 'max'),
          frequency=('InvoiceNo', 'nunique'),
          monetary=('Revenue', 'sum')
      )
      .reset_index()
)

customer_df.head()


,CustomerID,last_purchase_date,frequency,monetary
0,12346,2011-01-18 10:01:00,1,77183.60
1,12347,2011-12-07 15:52:00,7,4310.00
2,12348,2011-09-25 13:13:00,4,1797.24
3,12349,2011-11-21 09:51:00,1,1757.55
4,12350,2011-02-02 16:01:00,1,334.40


In [8]:
# Lets calculate recency in days
customer_df['recency_days'] = (
    reference_date - customer_df['last_purchase_date']
).dt.days


In [9]:
CHURN_THRESHOLD = 180

customer_df['is_churned'] = (
    customer_df['recency_days'] > CHURN_THRESHOLD
).astype(int)


In [10]:
customer_df['is_churned'].value_counts(normalize=True) * 100


is_churned
0    80.2213
1    19.7787
Name: proportion, dtype: float64

In [11]:
customer_df[customer_df['is_churned'] == 1].head()


,CustomerID,last_purchase_date,frequency,monetary,recency_days,is_churned
0,12346,2011-01-18 10:01:00,1,77183.6,325,1
4,12350,2011-02-02 16:01:00,1,334.4,309,1
6,12353,2011-05-19 17:47:00,1,89.0,203,1
7,12354,2011-04-21 13:11:00,1,1079.4,231,1
8,12355,2011-05-09 13:49:00,1,459.4,213,1


In [12]:
customer_df[customer_df['is_churned'] == 0].head()


,CustomerID,last_purchase_date,frequency,monetary,recency_days,is_churned
1,12347,2011-12-07 15:52:00,7,4310.00,1,0
2,12348,2011-09-25 13:13:00,4,1797.24,74,0
3,12349,2011-11-21 09:51:00,1,1757.55,18,0
5,12352,2011-11-03 14:37:00,8,2506.04,35,0
9,12356,2011-11-17 08:40:00,3,2811.43,22,0


In [13]:
# Save the data to a CSV file for modeling
customer_df.to_csv(
    "../data/processed/customer_churn_features.csv",
    index=False
)
